In [17]:
from transformers import AutoModelForCausalLM, GemmaConfig, AutoTokenizer, AutoModel, MistralConfig, MistralModel, MistralForCausalLM, LlamaConfig, LlamaForCausalLM
import torch
import torch.nn as nn
import torch.nn.init as init
import json
import pickle
import pandas as pd

In [18]:
input_ids = torch.load('/kaggle/input/tensor/tensor.pt')

/tmp/ipykernel_13/1273661524.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  input_ids = torch.load('/kaggle/input/tensor/tensor.pt')


In [19]:
input_ids

tensor([[ 2653,    42,  9355,  ...,  4202,  4391, 15817],
        [    0,     0,     0,  ...,     6,     9,  6885],
        [    0,     0,     0,  ...,     7,  2557,  6060],
        ...,
        [    0,     0,     0,  ..., 20936,  1081,  5880],
        [    0,     0,     0,  ..., 24459, 18840,  6060],
        [    0,     0,     0,  ..., 24459, 18840,  6060]])

In [20]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/tokenize")


In [21]:
config = LlamaConfig(
    vocab_size=len(tokenizer.vocab),       # Keep this standard or adjust for your use case
    hidden_size=256,        # Reduce from the standard size (e.g., 2048 or 1024)
    num_hidden_layers=6,   # Reduce the number of transformer layers
    num_attention_heads=8,  # Reduce the number of attention heads
    intermediate_size=512, # Adjust feed-forward layer size (standard is usually 4x hidden size)
    max_position_embeddings=256,  # Adjust as per use case
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
)

# Create the model with the adjusted configuration
model = LlamaForCausalLM(config)

In [22]:
# Load the model's state_dict from the file path
model_path = "/kaggle/input/model-77/model_epoch_2 (7).pth"
model.load_state_dict(torch.load(model_path))

/tmp/ipykernel_13/4244569867.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [23]:
import math

In [24]:
# Define the Russian prompt text
prompt_text = "Введите текст на русском языке здесь."

# Tokenize the prompt with attention mask
inputs = tokenizer(prompt_text, return_tensors="pt", padding=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Ensure model is in evaluation mode
model.eval()

# Define a function to calculate perplexity and print output text
def calculate_perplexity_and_output(input_ids, attention_mask, model):
    # Get model predictions and loss with attention mask
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss  # Cross-entropy loss

    # Calculate perplexity
    perplexity = math.exp(loss.item())

    # Generate model output text with pad_token_id set to eos_token_id
    output_ids = output_ids = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=50,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.7
)


    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return perplexity, output_text

# Calculate perplexity and generate output
perplexity, output_text = calculate_perplexity_and_output(input_ids, attention_mask, model)
print(f"Perplexity: {perplexity}")
print(f"Output Text: {output_text}")

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Perplexity: 574.050516104102
Output Text: Введите текст на русском языке здесь. В русском языке слово обозначены это только и в русском языке в русском языке — это книга даминана, который обозначают ацен-советы и его как правило — слово и слово «со


In [27]:
# Define the Russian prompt text
prompt_text = ['Расскажи мне о самых известных русских писателях.',
                'Опиши события Октябрьской революции.',
                'Какие популярные русские народные сказки ты знаешь',
                'Что ты можешь рассказать о культуре и традициях России',
                'Как развивалась русская живопись в веке',
                'Назови самые посещаемые туристические места в Москве',
                'Объясни, как функционирует российская политическая система',
                'Какие научные достижения сделали российские ученые',
                'Как празднуют Новый год в России',
                'Что такое балет и как он связан с Россией']

for x in prompt_text:
    # Tokenize the prompt with attention mask
    inputs = tokenizer(x, return_tensors="pt", padding=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Ensure model is in evaluation mode
    model.eval()

    # Define a function to calculate perplexity and print output text
    def calculate_perplexity_and_output(input_ids, attention_mask, model):
        # Get model predictions and loss with attention mask
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss  # Cross-entropy loss

        # Calculate perplexity
        perplexity = math.exp(loss.item())

        # Generate model output text with pad_token_id set to eos_token_id
        output_ids = output_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=50,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7
    )


        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        return perplexity, output_text

    # Calculate perplexity and generate output
    perplexity, output_text = calculate_perplexity_and_output(input_ids, attention_mask, model)
    print(f"Perplexity: {perplexity}")
    print(f"Output Text: {output_text}")

Perplexity: 758.797789466119
Output Text: Расскажи мне о самых известных русских писателях.В творчестве рассказа о жизни; с восемнадцати лет — некий вечер. Создал на сцене театрального театра; у него не очень много и не могли менять.Внезап
Perplexity: 175.21097679300226
Output Text: Опиши события Октябрьской революции.== Сведения о жизни ==Повесть о жизни:Акарити ИзабелевскаяАкаритри.Акасикотематика
Perplexity: 477.0364973487989
Output Text: Какие популярные русские народные сказки ты знаешь. На лице изображены три чёрные песни, как и тридца, а в каждом разновидности, в которых все и в свете.== Распространение ==Двух
Perplexity: 212.83465453822186
Output Text: Что ты можешь рассказать о культуре и традициях России на Украине в  году, когда ее брат Николай Иванович Ячинский был сослан из села Михайловское уезда в  году, в 
Perplexity: 324.00836979021864
Output Text: Как развивалась русская живопись в веке — в виде и мифах, иногда в его коллекциях и искусстве, и в ряде случаев — в искусс

In [28]:
import torch
import torch.nn as nn
# Tokenization

# Define training parameters
batch_size = 32
learning_rate = 3e-4
epochs = 1
perplexity_values = []

# Setup DataLoader for batching
train_data = torch.utils.data.TensorDataset(input_ids)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Optimizer and loss
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training loop with perplexity tracking and checkpoint saving
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    num_batches = len(train_loader)

    for batch_num, batch in enumerate(train_loader):
        inputs = batch[0].to(model.device)

        # Forward pass
        outputs = model(inputs, labels=inputs)
        loss = outputs.loss
        epoch_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Record perplexity at every 0.1 epoch and save checkpoint
        if batch_num % (num_batches // 10) == 0:
            perplexity = torch.exp(torch.tensor(epoch_loss / (batch_num + 1)))
            perplexity_values.append(perplexity.item())
            print(f"Perplexity at {epoch + (batch_num / num_batches):.1f} epoch: {perplexity.item()}")

            # Save model checkpoint
            checkpoint_path = f"./model_checkpoint_epoch_{epoch + 1 + 2}batch{batch_num + 1+2}.pth"
            torch.save({
                'epoch': epoch + 1,
                'batch_num': batch_num + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.item()
            }, checkpoint_path)
            print(f"Checkpoint saved at {checkpoint_path}")

            # Save model at the end of each epoch
            model_save_path = f"./model_epoch_{epoch + 2}.pth"
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved at {model_save_path}")

# Convert perplexity values into a DataFrame for easier visualization
df_perplexity = pd.DataFrame(perplexity_values, columns=['Perplexity'])
print(df_perplexity)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn